In [1]:
import string
import random
from PIL import Image, ImageOps, ImageDraw, ImageFont, ImageChops
import matplotlib.pyplot as plt
import numpy as np
import ipywidgets
from ipywidgets import interactive, HBox, VBox
from img_processing import preprocess_img, DITHER_MODES
from ascii_art_non_mono_utils import *
from ascii_art_non_mono_algorithms import *
from skimage import metrics

In [2]:
font_path = "fonts/OpenSans-Regular.ttf"
size = 16
font = ImageFont.truetype(font_path, size)

In [3]:
orig_img = Image.open("imgs/irad_grad.bmp").convert("L")
#orig_img = ImageOps.invert(orig_img)

print(orig_img.size)

(512, 512)


In [4]:
def i_preprocess_img(scale_factor, contrast, brightness, quantize_colors, eq, dither):
    proc_img = preprocess_img(orig_img, 0.3, 1.1, 0.9, 0.2, 128, DITHER_MODES.NONE)
    plt.imshow(proc_img, cmap='gray', vmin=0, vmax=255, interpolation='none')

    plt.figure(figsize=(9, 3))
    plt.xticks([x for x in range(0, len(proc_img.histogram()), 25)])
    plt.bar([x for x in range(0, len(proc_img.histogram()))], proc_img.histogram())
    plt.grid()
    plt.show()

    return proc_img

interactive_preprocess = interactive(i_preprocess_img,
            scale_factor=(0.01, 1, 0.01),
            contrast=(0, 2, 0.01),
            brightness=(0, 2, 0.01),
            eq=(0, 1, 0.01),
            quantize_colors=(1, 256, 1),
            dither=DITHER_MODES)

controls = VBox(interactive_preprocess.children[:-1])
#display(HBox((controls, interactive_preprocess.children[-1])))

In [5]:
proc_img = interactive_preprocess.result

In [6]:
palette = list(filter(lambda a: a.isprintable(), string.printable))
print(len(palette))
print(palette)

95
['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', ':', ';', '<', '=', '>', '?', '@', '[', '\\', ']', '^', '_', '`', '{', '|', '}', '~', ' ']


In [7]:
all_lines = split_lines(proc_img, palette, font)
lines = [all_lines[len(all_lines) // 4], all_lines[len(all_lines) // 2], all_lines[-len(all_lines) // 4]]
for l in lines:
    display(l)

In [12]:
for l in lines:
    text_arr = generate_greedy_line(l, palette, font)
    print(evaluate_text_arr(text_arr, l, font))
    bg_img, bg_draw = new_img_draw(l.size)
    draw_text_arr(bg_draw, text_arr, font)
    display(ImageChops.difference(bg_img, l))

0.01553996396460135


0.011340129822295921


0.010590135990326589


In [14]:
for l in lines:
    lhs = lazy_generate_harmony_line(l, palette, font, 10, 0.8, 0.3, len(palette)//2)
    for i in range(5001):
        p_ids_arr, fit = next(lhs)
        if i % 1000 == 0:
            print(fit)
    bg_img, bg_draw = new_img_draw(l.size)
    draw_text_arr(bg_draw, palette_id_arr_to_text_arr(p_ids_arr, palette), font)
    display(ImageChops.difference(bg_img, l))

0.011940547480459486
0.01483847240598599
0.01483847240598599
0.015383196939050095
0.015452779095677894
0.015452779095677894


0.007384469292948466
0.009500030735335206
0.009500030735335206
0.009500030735335206
0.0095839372169439
0.00965891808703984


0.005328453141490692
0.008467362062657208
0.008671144623224089
0.008735648220232168
0.00915365702292034
0.009156279431423555


In [9]:
# text = greedy_algorithm(proc_img, palette, font)

# print(''.join(text))

# bg_img, draw = new_img_draw(proc_img.size)
# draw_text_arr(draw, text, font)
# print(evaluate_text_arr(text, proc_img, font))
# ImageChops.add(proc_img, bg_img)

In [10]:
# lhs = lazy_harmony_search(proc_img, palette, font, 5, 0.9, 0.3, len(palette)//2)

# for i in range(4000):
#     text = next(lhs)
#     if i % 1000 == 0:
#         print(evaluate_text_arr(text, proc_img, font))

# text = next(lhs)
# print(''.join(text))

# bg_img, draw = new_img_draw(proc_img.size)
# draw_text_arr(draw, text, font)
# print(evaluate_text_arr(text, proc_img, font))
# ImageChops.add(proc_img, bg_img)